<a href="https://colab.research.google.com/github/i2mmmmm/kaggle_flood/blob/main/catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME']='mmmmmlee'
os.environ['KAGGLE_KEY']='b4527f98e27813c6dae3fb199096dcb5'

In [2]:
!kaggle competitions download -c Playground-series-s4e5

 86% 24.0M/28.0M [00:00<00:00, 135MB/s] 
100% 28.0M/28.0M [00:00<00:00, 133MB/s]


In [3]:
!unzip 'Playground-series-s4e5.zip'

Archive:  Playground-series-s4e5.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

전처리

In [6]:
# 표준화
scaler = StandardScaler()
X = train.drop(columns=['FloodProbability', 'id'])
y = train['FloodProbability']
X_scaled = scaler.fit_transform(X)


In [ ]:
# 선형회귀
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('R2 Score:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

R2 Score: 0.8448773362840329
MSE: 0.000403206587090558


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=35)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('R2 Score:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

R2 Score: 0.8449835489068076
MSE: 0.00040395088979260915


In [ ]:

X_train = train.drop(columns=['FloodProbability', 'id'])
y_train = train['FloodProbability']
X_test = test.drop(columns=['id'])

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression

# 모델 학습
model_lr = LinearRegression()
model_lr.fit(X_train_scaled, y_train)

# 예측
y_pred_lr = model_lr.predict(X_test_scaled)

# 결과 출력
test['FloodProbability'] = y_pred_lr
print(test[['id', 'FloodProbability']])

             id  FloodProbability
0       1117957          0.573615
1       1117958          0.455243
2       1117959          0.454742
3       1117960          0.466204
4       1117961          0.466068
...         ...               ...
745300  1863257          0.477227
745301  1863258          0.449369
745302  1863259          0.624330
745303  1863260          0.550925
745304  1863261          0.511472

[745305 rows x 2 columns]


In [ ]:
# 출력
test[['id', 'FloodProbability']].to_csv('submission.csv', index=False)

# 첫번째 제출 baseline 0.84458

In [ ]:
sample_submission

,id,FloodProbability
0,1117957,0.5
1,1117958,0.5
2,1117959,0.5
3,1117960,0.5
4,1117961,0.5
...,...,...
745300,1863257,0.5
745301,1863258,0.5
745302,1863259,0.5
745303,1863260,0.5


In [ ]:
X = train.drop(columns=['FloodProbability', 'id'])
y = train['FloodProbability']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=35)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

model_lgb = lgb.LGBMRegressor()
model_lgb.fit(X_train_scaled, y_train)
y_pred_lgb = model_lgb.predict(X_val_scaled)

print('LightGBM R2 Score:', r2_score(y_val, y_pred_lgb))
print('LightGBM MSE:', mean_squared_error(y_val, y_pred_lgb))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.615218 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 369
[LightGBM] [Info] Number of data points in the train set: 894365, number of used features: 20
[LightGBM] [Info] Start training from score 0.504465
LightGBM R2 Score: 0.7665943094099801
LightGBM MSE: 0.0006082221321130316


In [ ]:
import xgboost as xgb

model_xgb = xgb.XGBRegressor()
model_xgb.fit(X_train_scaled, y_train)
y_pred_xgb = model_xgb.predict(X_val_scaled)

print('XGBoost R2 Score:', r2_score(y_val, y_pred_xgb))
print('XGBoost MSE:', mean_squared_error(y_val, y_pred_xgb))

XGBoost R2 Score: 0.81016914113262
XGBoost MSE: 0.0004946722996740113


In [9]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 6.7 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor

model_cat = CatBoostRegressor(verbose=0)
model_cat.fit(X_train_scaled, y_train)
y_pred_cat = model_cat.predict(X_val_scaled)

print('CatBoost R2 Score:', r2_score(y_val, y_pred_cat))
print('CatBoost MSE:', mean_squared_error(y_val, y_pred_cat))

CatBoost R2 Score: 0.8463552061638621
CatBoost MSE: 0.0004003765454854714


In [14]:
from sklearn.preprocessing import StandardScaler

# train 가지고 성능테스트
scaler = StandardScaler()
X = train.drop(columns=['FloodProbability', 'id'])
y = train['FloodProbability']

# 스케일링
scaler = StandardScaler()

# train만
X_scaled = scaler.fit_transform(X)




# train, test
X_train = train.drop(columns=['FloodProbability', 'id'])
y_train = train['FloodProbability']
X_test = test.drop(columns=['id'])

# 스케일링
scaler = StandardScaler()

# train,test 전부
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [19]:
best_params = {'learning_rate': 0.1, 'l2_leaf_reg': 5, 'iterations': 1000, 'depth': 8}
model_cat_best = CatBoostRegressor(**best_params)

# 모델 학습
model_cat_best.fit(X_train_scaled, y_train)

# 테스트 데이터에 대한 예측
y_test_pred = model_cat_best.predict(X_test_scaled)

# 결과 출력
test2['FloodProbability'] = y_test_pred
print(test2[['id', 'FloodProbability']])

0:	learn: 0.0500571	total: 337ms	remaining: 5m 36s
1:	learn: 0.0490890	total: 614ms	remaining: 5m 6s
2:	learn: 0.0481843	total: 883ms	remaining: 4m 53s
3:	learn: 0.0473314	total: 1.14s	remaining: 4m 44s
4:	learn: 0.0465012	total: 1.4s	remaining: 4m 37s
5:	learn: 0.0457188	total: 1.65s	remaining: 4m 34s
6:	learn: 0.0449551	total: 1.92s	remaining: 4m 32s
7:	learn: 0.0442362	total: 2.17s	remaining: 4m 29s
8:	learn: 0.0435653	total: 2.57s	remaining: 4m 42s
9:	learn: 0.0429041	total: 3s	remaining: 4m 57s
10:	learn: 0.0422678	total: 3.44s	remaining: 5m 9s
11:	learn: 0.0416467	total: 3.88s	remaining: 5m 19s
12:	learn: 0.0410560	total: 4.27s	remaining: 5m 24s
13:	learn: 0.0405015	total: 4.67s	remaining: 5m 29s
14:	learn: 0.0399493	total: 5.08s	remaining: 5m 33s
15:	learn: 0.0394146	total: 5.49s	remaining: 5m 37s
16:	learn: 0.0388838	total: 5.91s	remaining: 5m 41s
17:	learn: 0.0383820	total: 6.39s	remaining: 5m 48s
18:	learn: 0.0379047	total: 6.72s	remaining: 5m 46s
19:	learn: 0.0374346	total: 

In [21]:
# 출력
test2[['id', 'FloodProbability']].to_csv('submission.csv', index=False)

두번째 제출 - catboost 0.846

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# 선형 회귀 모델 학습
model_lr = LinearRegression()
model_lr.fit(X_scaled, y)
y_pred_lr = model_lr.predict(X_scaled)

# 앙상블 예측 (가중 평균)
ensemble_pred = (0.5 * y_pred_cat_best) + (0.5 * y_pred_lr)

# 성능 평가
print('Ensemble R2 Score:', r2_score(y, ensemble_pred))
print('Ensemble MSE:', mean_squared_error(y, ensemble_pred))

Ensemble R2 Score: 0.8486498093828362
Ensemble MSE: 0.0003940644486519777


In [ ]:
X_train = train.drop(columns=['FloodProbability', 'id'])
y_train = train['FloodProbability']
X_test = test.drop(columns=['id'])

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model_cat = CatBoostRegressor(verbose=0)
model_cat.fit(X_train_scaled, y_train)

y_pred_cat = model_cat.predict(X_test_scaled)


# 모델 학습
model_lr = LinearRegression()
model_lr.fit(X_train_scaled, y_train)

# 예측
y_pred_lr = model_lr.predict(X_test_scaled)



In [ ]:
# CatBoost와 선형 회귀 앙상블

# 선형 회귀 모델 학습
model_lr = LinearRegression()
model_lr.fit(X_train_scaled, y_train)
y_pred_lr = model_lr.predict(X_val_scaled)

# 앙상블 예측 (가중 평균)
ensemble_pred = (0.5 * y_pred_cat) + (0.5 * y_pred_lr)

# 성능 평가
print('Ensemble R2 Score:', r2_score(y_val, ensemble_pred))
print('Ensemble MSE:', mean_squared_error(y_val, ensemble_pred))